# Трансформери

# Дел 1: Едноставен Seq2Seq модел и модел со додаден attention



##Едноставен Seq2Seq модел


Целта на првиот дел од оваа вежба е да направиме превод од еден во друг јазик. 

Во случајот ќе правиме превод од Македонски во Англиски јазик.

Во следниот дел се дадени неколку imports кои ќе ги користиме.

[PyTorch](https://pytorch.org/) е рамка за машинско учење (ML framework) со чија помош можеме да ја искористиме целосната моќ на стартување со помош на GPU, наместо CPU. 

Целта во случајот ни е користејќи тензори да ги искористиме можностите на GPU, и да добиеме околу 50 пати побрзо процесирање.

Во рамки на Colab notebook можеме лесно да го избереме ова, во делот Runtime -> Change runtime type.

In [3]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import pandas as pd
import os

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Податочното множество кое ќе го користиме може да го пронајдете и симнете од [овој линк](https://www.manythings.org/anki/). Потоа, можете да го ставите на вашиот Drive и да го користите на тој начин, или пак наједноставно да го ставите во самиот Colab notebook преку Files -> Upload to session storage.

In [4]:
os.rename("mkd.txt", "mkd-eng.txt")

Секој збор ќе го претставиме како one-hot вектор, односно вектор со многу 0 и еден 1, што значи истиот ќе има голема димензионалност. 
За секој збор ќе ни треба еден уникатен индекс, за понатамошната работа. За да може лесно да ги искористиме понатаму, користиме помошна класа Lang која има два речници (dictionaries), еден со mapping word->index, друг со mapping index->word.
Дополнително, користиме пребројување на секој збор за да можеме понатаму да ги замениме ретките зборови.

In [5]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        # Here we count the SOS and EOS
        self.n_words = 2 

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

Бидејќи фајлот кој го користиме има кирилични симболи мораме да го задржиме во Unicode encoding.

Дополнително, ќе направиме се да биде lowercase (само со мали букви) и ќе ги отстраниме интерпукциските знаци.


In [6]:
def normalizeString(s):
    s = s.lower().strip()
    s = re.sub(r"([.!?])", r" \1", s)
    return s

За да ги прочитаме податоците ќе го поделиме фајлот на линии (редови) и потоа ќе ги поделиме и самите редови. Бидејќи фајлот е даден во формат Англиски јазик -> Македонски јазик, ќе ги превртиме паровите за да можеме да правиме превод од Македонски во Англиски јазик.


In [7]:
def readLangs(lang1, lang2, reverse=False):
    # Read the file and split into lines
    lines = open('%s-%s.txt' % (lang1, lang2), encoding='utf-8').read().strip().split('\n')

    pairs = []

    print(len(lines))

    for line in lines:
      tmp = []
      s = re.split('\t', line)
      s0 = normalizeString(s[0])
      s1 = normalizeString(s[1])
      tmp.append(s0)
      tmp.append(s1)
      pairs.append(tmp)

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [8]:
MAX_LENGTH = 100

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

Подготовката на податоците, во целост, се состои од:


> Читање на фајлот и поделбата на истиот по редови (линии) и понатаму поделба на секој ред по парови

> Нормализација на текстот, филтрирање по содржина и должина

> Креирање на листите од парови на зборови







In [9]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('mkd', 'eng', True)
print(random.choice(pairs))

8040
Read 8040 sentence pairs
Trimmed to 91 sentence pairs
Counting words...
Counted words:
eng 120
mkd 99
['среќен е .', 'he is happy .']


RRN мрежа е рекурентна невронска мрежа која работи врз секвенца и го користи сопствениот output како input во следните чекори.

Како што видовме во рамки на предавањата, еден Seq2Seq модел може најлесно да се претстави како модел на две RNN мрежи, едната играјќи ја улогата на енкодер, а другата играјќи ја улогата на декодер. 

Енкодерот за секој еден input збор генерира output: вектор за самиот збор и hidden state кој се користи за следниот input збор.

In [10]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

Декодерот го користи context векторот (последниот output од енкодерот) како 
initial hidden state.

In [11]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

##Seq2Seq модел со додаден attention


Како што видовме од предавањата, можеме да додадеме attention кон декодерот, за да таа мрежа може да се фокусира на различен дел од output-от на енкодерот во секој еден негов чекор.

Ова се прави со помош на attention weights, со чија помош доаѓаме до weighted combination (се множат тежините со output векторите на енкодерот).

Имплементацијата на овој декодер е дадена во продолжение.

In [12]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

##Тренирање на моделот

За да можеме да го тренираме моделот, најпрво ги подготвуваме податоците. 
За секој пар ни е потребен еден input тензор (индекси на зборовите во input реченицата) и еден target тензор (индекси на зборовите во target реченицата).

In [13]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

“Teacher forcing” е концептот на користење на вистински target outputs за секој следен input, на место на користење на предикцијата на декодерот. Овај концепт ни овозможва побрза конвергенција, но може да предизвика и нестабилност.

Менувајќи го teacher_forcing_ratio можеме да го користиме концептот повеќе или помалку, или пак воопшто да не го користиме овој концепт.

In [14]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # Detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

Во прилог е помошна функција која ќе ни отпечати колку време е поминато во извршување, колку е естимираното преостанато време и колкав прогрес е направен до тој момент на тренирање.


In [15]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

Целосниот процес на тренирање може да го поделиме на неколку чекори:

> Старт на тајмерот

> Иницијализација на критериумите и optimizers

> Креирање на множество на тренинг парови

> Креирање на низа на losses за понатамошна претстава

> Повикување на train многу пати и повремено печатење на прогрес


In [16]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # This locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [17]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

##Оценка на моделот

Оцената е генерално иста како и тренирањето, со тоа што во овој дел немаме targets, туку едноставно предикциите на декодерот зависат само од неговите претходни предикции. 

In [18]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

Креираме функција за полесна оцена на random реченици од тренинг множествто.


In [19]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

Бидејќи ги имаме сите потребни помошни функции, во овој момент можеме да ја иницијализираме мрежата и да го почнеме тренирањето.

Бидејќи направивме филтрирање и немаме премногу податоци, релативно мала мрежа со 256 скриени јазли и еден GPU слој би ни бил доволен.
Очекувајте слични временски инстанци доколку го извршувате кодот на Colab, но може и поголеми доколку го извршувате локално.

In [20]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 75000, print_every=5000)

1m 38s (- 22m 53s) (5000 6%) 0.6731
3m 8s (- 20m 24s) (10000 13%) 0.0222
4m 39s (- 18m 36s) (15000 20%) 0.0064
6m 9s (- 16m 57s) (20000 26%) 0.0036
7m 38s (- 15m 17s) (25000 33%) 0.0025
9m 8s (- 13m 42s) (30000 40%) 0.0019
10m 37s (- 12m 8s) (35000 46%) 0.0015
12m 5s (- 10m 34s) (40000 53%) 0.0013
13m 33s (- 9m 2s) (45000 60%) 0.0011
15m 3s (- 7m 31s) (50000 66%) 0.0009
16m 32s (- 6m 1s) (55000 73%) 0.0008
18m 2s (- 4m 30s) (60000 80%) 0.0007
19m 31s (- 3m 0s) (65000 86%) 0.0007
20m 59s (- 1m 29s) (70000 93%) 0.0006
22m 27s (- 0m 0s) (75000 100%) 0.0006


In [21]:
evaluateRandomly(encoder1, attn_decoder1)

> тој е болен .
= he is sick .
< he is sick . <EOS>

> таа не е мажена .
= she isn't married .
< she isn't married . <EOS>

> среќен сум .
= i am happy .
< i am happy . <EOS>

> на час се .
= they are in class .
< they are in class . <EOS>

> тука сме .
= we are here .
< we are here . <EOS>

> малку сум тажен .
= i am a trifle sad .
< i am a trifle sad . <EOS>

> јас сум момче .
= i am a boy .
< i am a boy . <EOS>

> тој е диск-џокеј .
= he is a dj .
< he is a dj . <EOS>

> увилен сум .
= i am woebegone .
< i am woebegone . <EOS>

> ние сме мажи .
= we are men .
< we are men . <EOS>



Како и до сега, можеме да искористиме претставување на податоците. Во случајот ќе го прикажеме attention output како матрица, каде редици се output чекорите, а колони се input чекорите.

In [23]:
output_words, attentions = evaluate(
    encoder1, attn_decoder1, 'на час се .')
plt.matshow(attentions.numpy())

In [24]:
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(
        encoder1, attn_decoder1, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions)


evaluateAndShowAttention("го мразат .")

input = го мразат .
output = he is hated . <EOS>


# Дел 2: Користење на Huggingface Transformers

Huggingface е open-source provider за NLP технологии (технологии за процесирање на природни јазици). 

Huggingface transformers е пакет на популарни Python библиотеки кои овозможуваат користење на различни претренирани модели кои ни се од неверојатна помош при различни NLP задачи.

Во рамки на овој дел од вежбата ќе ги искористиме имплементациите на два познати модели: BERT и GPT.

Доколку сакате да погледнете документација како да ги користите сите модели кои ги нуди Huggingface истото може да го направите преку [овој линк](https://huggingface.co/transformers/pretrained_models.html).

Во рамки на оваа вежба фокусот е на овие два модели. Во следната вежба фокусот е на задачите кои моделите кои ги нуди Huggingface Transformers може да ги извршат.

In [25]:
! pip install transformers

     |████████████████████████████████| 3.1 MB 5.4 MB/s 
     |████████████████████████████████| 59 kB 6.2 MB/s 
     |████████████████████████████████| 596 kB 18.2 MB/s 
     |████████████████████████████████| 895 kB 40.3 MB/s 
     |████████████████████████████████| 3.3 MB 22.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Во следните два примери ќе користиме AutoClasses, за кои архитектурата е одредена од името или патеката до претренираниот модел кој се специфицира преку from_pretrained() методот.

На овој начин го имаме релевантниот модел автоматски.





## BERT модел

BERT моделот во целост можете да го погледнете на [овој линк](https://github.com/google-research/bert). 

За овој модел може да погледнете повеќе детали на [овој линк](https://openai.com/blog/better-language-models/).

BERT (Bidirectional Encoder Representations from Transformers) моделот е дизајниран да работи со нелабелиран текст и е еден од најдобрите NLP модели кој може да се искористи за најразлични задачи кои ќе ги погледнеме во оваа и следната вежба.

Овој модел е воведен во [овој труд](https://arxiv.org/abs/1810.04805).



Во случајот ќе направиме класификација на реченица, во однос на друга реченица, дали истата е парафраза или не.

Најпрво, избираме да користиме Bert модел, во случајот cased и finetuned. 
Потоа, за да направиме секвенца од две реченици го користиме encode_plus методот. 

Ја даваме ваквата реченица на моделот и истата ќе биде класифицирана како 1 (е парафраза) или 0 (не е парафраза).

Можеме дополнително и да пресметаме softmax на резултатот за да ги добиеме веројатностите за припаѓање кон одредената класа.

In [26]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased-finetuned-mrpc")

classes = ["not paraphrase", "is paraphrase"]

sequence_0 = "The company HuggingFace is based in the United States"
sequence_1 = "Apples are especially great for you"
sequence_2 = "HuggingFace's headquarters are situated in the USA"

paraphrase = tokenizer.encode_plus(sequence_0, sequence_2, return_tensors="pt")
not_paraphrase = tokenizer.encode_plus(sequence_0, sequence_1, return_tensors="pt")

paraphrase_classification_logits = model(**paraphrase)[0]
not_paraphrase_classification_logits = model(**not_paraphrase)[0]

paraphrase_results = torch.softmax(paraphrase_classification_logits, dim=1).tolist()[0]
not_paraphrase_results = torch.softmax(not_paraphrase_classification_logits, dim=1).tolist()[0]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

In [27]:
print("Should be paraphrase")
for i in range(len(classes)):
    print(f"{classes[i]}: {round(paraphrase_results[i] * 100)}%")

print("\nShould not be paraphrase")
for i in range(len(classes)):
    print(f"{classes[i]}: {round(not_paraphrase_results[i] * 100)}%")

Should be paraphrase
not paraphrase: 6%
is paraphrase: 94%

Should not be paraphrase
not paraphrase: 95%
is paraphrase: 5%


## GPT модел

GPT (Generative Pre-trained Transformer) моделите се доста моќни модели кои може да извршат најголем дел од NLP задачите без потреба од тренинг, а со одлични перформанси.

Доколку сакате да прочитате повеќе за овие модели тоа можете да го направите на следните линкови:


> [GPT-1](https://cdn.openai.com/research-covers/language-unsupervised/language_understanding_paper.pdf)


> [GPT-2](https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf)


> [GPT-3](https://arxiv.org/pdf/2005.14165.pdf)


Во нашиот случај ќе ја извршиме истата задача како и во претходниот пример, за да ги споредиме двата модели.

Можеме да воочиме дека за оваа задача овој модел дава подобра точност.

In [28]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForSequenceClassification.from_pretrained("gpt2")

classes = ["not paraphrase", "is paraphrase"]

sequence_0 = "The company HuggingFace is based in the United States"
sequence_1 = "Apples are especially great for you"
sequence_2 = "HuggingFace's headquarters are situated in the USA"

paraphrase = tokenizer.encode_plus(sequence_0, sequence_2, return_tensors="pt")
not_paraphrase = tokenizer.encode_plus(sequence_0, sequence_1, return_tensors="pt")

paraphrase_classification_logits = model(**paraphrase)[0]
not_paraphrase_classification_logits = model(**not_paraphrase)[0]

paraphrase_results = torch.softmax(paraphrase_classification_logits, dim=1).tolist()[0]
not_paraphrase_results = torch.softmax(not_paraphrase_classification_logits, dim=1).tolist()[0]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print("Should be paraphrase")
for i in range(len(classes)):
    print(f"{classes[i]}: {round(paraphrase_results[i] * 100)}%")

print("\nShould not be paraphrase")
for i in range(len(classes)):
    print(f"{classes[i]}: {round(not_paraphrase_results[i] * 100)}%")